In [1]:
import torch
from torch.utils.data import Dataset, DataLoader

# 1. Create a Custom Dataset
class CustomDummyDataset(Dataset):
    def __init__(self, num_samples=100, num_features=5):
        self.data = torch.randn(num_samples, num_features)  # Dummy input features
        self.labels = torch.randint(0, 2, (num_samples, 1)).float() # Dummy binary labels (0 or 1)

    def __len__(self):
        return len(self.data) # Total number of samples

    def __getitem__(self, idx):
        # Return one sample (features and its corresponding label)
        return self.data[idx], self.labels[idx]

# 2. Instantiate the Dataset
my_dataset = CustomDummyDataset(num_samples=1000, num_features=5)
print(f"Total samples in dataset: {len(my_dataset)}")

# Get a single sample directly (just for demonstration, DataLoader handles this usually)
sample_features, sample_label = my_dataset[0]
print(f"First sample features: {sample_features}, label: {sample_label}")

# 3. Create a DataLoader
# batch_size: how many samples per batch
# shuffle: True means shuffle the data at the beginning of each epoch (training pass)
# num_workers: how many separate processes to use for data loading (0 means main process)
batch_size = 32
train_dataloader = DataLoader(my_dataset, batch_size=batch_size, shuffle=True, num_workers=0)

# 4. Iterate through the DataLoader to get batches
print(f"\nIterating through DataLoader with batch_size={batch_size}:")
for i, (features_batch, labels_batch) in enumerate(train_dataloader):
    print(f"Batch {i+1}: Features shape {features_batch.shape}, Labels shape {labels_batch.shape}")
    if i == 2: # Just print the first 3 batches to avoid too much output
        break

print("\nDataLoader will automatically handle batching, shuffling, and memory management!")

Total samples in dataset: 1000
First sample features: tensor([-0.2876, -0.9066,  0.8859,  0.2278,  1.0939]), label: tensor([0.])

Iterating through DataLoader with batch_size=32:
Batch 1: Features shape torch.Size([32, 5]), Labels shape torch.Size([32, 1])
Batch 2: Features shape torch.Size([32, 5]), Labels shape torch.Size([32, 1])
Batch 3: Features shape torch.Size([32, 5]), Labels shape torch.Size([32, 1])

DataLoader will automatically handle batching, shuffling, and memory management!


In [2]:
# Assuming you have MySimpleRegressor, CustomDummyDataset, and DataLoader defined from previous steps
# (You'll put this in your 02_PyTorch_Data_Prep_and_Training_Loop.ipynb)

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

# --- Re-define our Dataset and Model (for completeness in this single code block) ---

class CustomDummyDataset(Dataset):
    def __init__(self, num_samples=100, num_features=5):
        # Let's adjust this for a simple regression problem: features and a continuous target
        self.data = torch.randn(num_samples, num_features)
        # Create labels as a simple linear relationship + some noise
        self.labels = (self.data[:, 0] * 2 + self.data[:, 1] * 0.5 + 1.0 + torch.randn(num_samples) * 0.1).unsqueeze(1)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]

class MySimpleRegressor(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(MySimpleRegressor, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        return x

# --- Configuration for our training ---
input_dim = 5
hidden_dim = 10
output_dim = 1
learning_rate = 0.01
epochs = 50 # How many times we'll go through the entire dataset
batch_size = 32
num_samples = 1000

# 1. Instantiate the Dataset and DataLoader
dataset = CustomDummyDataset(num_samples=num_samples, num_features=input_dim)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# 2. Instantiate the Model
model = MySimpleRegressor(input_dim, hidden_dim, output_dim)

# 3. Define Loss Function (for regression, we use MSE Loss)
loss_fn = nn.MSELoss()

# 4. Define Optimizer (Adam is a good general-purpose choice)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Check if GPU is available and move model to GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
print(f"Using device: {device}")


# --- The Training Loop ---
print("\nStarting Training...")
for epoch in range(epochs):
    # 'train_loss' will accumulate loss for this epoch
    train_loss = 0.0

    # Iterate over batches of data from the DataLoader
    for batch_idx, (inputs, targets) in enumerate(dataloader):
        # Move inputs and targets to the same device as the model (GPU if available)
        inputs, targets = inputs.to(device), targets.to(device)

        # 1. Forward Pass: Make a prediction
        predictions = model(inputs)

        # 2. Calculate Loss
        loss = loss_fn(predictions, targets)
        train_loss += loss.item() # Add loss to accumulator for logging

        # 3. Zero the Gradients: Clear old gradients
        optimizer.zero_grad()

        # 4. Backward Pass: Compute gradients
        loss.backward()

        # 5. Optimizer Step: Update model weights
        optimizer.step()

    # Calculate average loss for the epoch
    avg_train_loss = train_loss / len(dataloader)

    # Print progress (optional, but good for tracking)
    if (epoch + 1) % 10 == 0 or epoch == 0: # Print every 10 epochs and first epoch
        print(f"Epoch [{epoch+1}/{epochs}], Average Loss: {avg_train_loss:.4f}")

print("\nTraining Finished!")

Using device: cuda

Starting Training...
Epoch [1/50], Average Loss: 2.8987
Epoch [10/50], Average Loss: 0.0250
Epoch [20/50], Average Loss: 0.0150
Epoch [30/50], Average Loss: 0.0129
Epoch [40/50], Average Loss: 0.0123
Epoch [50/50], Average Loss: 0.0122

Training Finished!
